# Indigo INDY Staking Rewards Calx
**The below note book is useful to calcuate INDY and ADA rewards distributed to INDY stakers of the Indigo Protocol. Indigo Protocol currently distributes ~6046 INDY per epoch on a pro-rata basis. Stakers are also rewarded with a part of ADA fees generated by the protocol. From v2 onwards, stakers are also awarded 30% of the ADA interested revenue generated by the protocol. These Stakers are an integral part of the protocol and can use this staked INDY to vote in on-chain voting. The Staking APR for these INDY rewards is reported on a per epoch basis and Staking ADA rewards is reported on a 3-months rolling basis and is displyed on the official Indigo Protocol dashboard.**

In [12]:
import os
import pandas as pd
import time
from datetime import datetime
from typing import Final
import glob
import calendar, time;
import urllib.request
import json

**The total INDY staked eligible for INDY staking rewards is as follows:**

In [13]:
REF_DATE: Final[datetime] = datetime(2022, 11, 27)
EPOCH: Final[int] = 379

def get_difference(date: datetime) -> int:
    difference = date - REF_DATE

    return difference.days

path: str = os.getcwd() + "\\Staking_Data"
# path: str = os.getcwd() + "\\test"
# print(path)

if "Staking_Data_Cleaned" not in os.listdir(os.getcwd()):
    os.mkdir(f"{os.getcwd()}\\Staking_Data_Cleaned")

timestamps: list[int] = []
days = []

for filename in os.listdir(path):
    
#     print(filename)
    filename = filename.replace(".", "-")
    fname: list[str] = filename.split("-")
    timestamps.append(int(fname[-2]))

for timestamp in timestamps:
    full_date = time.strftime("%d %b %Y", time.gmtime(timestamp))
    date = pd.to_datetime(full_date)
    diff = get_difference(date)
    days.append(diff)

for index, file in list(enumerate(os.listdir(path))):
    df = pd.read_csv(f"{path}\\{file}")

    df.insert(0, column="epoch", value= 379 + (days[index])//5 )
    df.to_csv(f"{os.getcwd()}\\Staking_Data_Cleaned\\{timestamps[index]}.csv")
#     print(file)

# # Get CSV files list from a folder
path1 = "Staking_Data_Cleaned"
csv_files = glob.glob(path1 + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list1 = (pd.read_csv(file) for file in csv_files)
# Concatenate all DataFrames
staking_df   = pd.concat(df_list1, ignore_index=True)
staking_df.insert(4, column="Actual_INDY_Staked", value=staking_df['indy_staked']/1000000)
staking_df=staking_df.drop("Unnamed: 0",axis=1)
columns_to_drop = ['owner']

# Drop the specified columns
dk = staking_df.drop(columns=columns_to_drop).groupby('epoch').sum()
dk=dk.reset_index()
print(dk.to_string())

     epoch    indy_staked  Actual_INDY_Staked
0      380   252235251858        2.522353e+05
1      381   338183054489        3.381831e+05
2      382   403064633504        4.030646e+05
3      383   430959309830        4.309593e+05
4      384   448986900680        4.489869e+05
5      385   502047226284        5.020472e+05
6      386   547405830558        5.474058e+05
7      387   614138594820        6.141386e+05
8      388   719188684531        7.191887e+05
9      389   804093171625        8.040932e+05
10     390   808820233225        8.088202e+05
11     391   794981320706        7.949813e+05
12     392  1206552094010        1.206552e+06
13     393  1068295334343        1.068295e+06
14     394  1058979515326        1.058980e+06
15     395  1096453450490        1.096453e+06
16     396  1155370340182        1.155370e+06
17     397  1192872517599        1.192873e+06
18     398  1252471612464        1.252472e+06
19     399   707311526785        7.073115e+05
20     400   673395530527        6

**Total INDY rewards distributed of Stakers in all the epochs till date is as follows:**


In [14]:
path: str = os.getcwd() + "\\INDY_Rewards"
    
excel_files = [file for file in os.listdir(path) if file.endswith('.xlsx')]
reward_df = pd.concat((pd.read_excel(os.path.join(path, file), engine='openpyxl') for file in excel_files), ignore_index=True)

columns_to_drop = ['addr','Expiration','AvailableAt']
reward_df = reward_df.drop(columns=columns_to_drop)
reward_df['amount']=reward_df['amount']/1000000


indexAge = reward_df[ (reward_df['purpose'] != 'INDY staking reward') ].index
reward_df.drop(indexAge , inplace=True)

reward_df=reward_df.groupby(['epoch','date','purpose'])


filtered_df1_month=reward_df.sum()
gk3 = filtered_df1_month.reset_index()
print(gk3.to_string())


    epoch       date              purpose       amount
0     452 2023-12-01  INDY staking reward  2398.000003
1     453 2023-12-06  INDY staking reward  2398.000008
2     454 2023-12-11  INDY staking reward  2398.000008
3     455 2023-12-16  INDY staking reward  2397.999984
4     456 2023-12-21  INDY staking reward  2398.000004
5     457 2023-12-26  INDY staking reward  2398.000005
6     458 2023-12-31  INDY staking reward  2397.999988
7     459 2024-01-05  INDY staking reward  2398.000008
8     460 2024-01-10  INDY staking reward  2397.999999
9     461 2024-01-15  INDY staking reward  2398.000002
10    462 2024-01-20  INDY staking reward  2397.999987
11    463 2024-01-25  INDY staking reward  2398.000002
12    464 2024-01-30  INDY staking reward  2398.000007
13    465 2024-02-04  INDY staking reward  2398.000001
14    466 2024-02-09  INDY staking reward  2398.000001
15    467 2024-02-14  INDY staking reward  2397.999988
16    468 2024-02-19  INDY staking reward  2397.999991
17    469 

**The INDY rewards APR based on the selected epoch is as below:**

In [15]:
# epoch

epoch1=[512]#change this
APY3_arr=[]
running_sum1=0
gk3= gk3.reset_index()
dk= dk.reset_index()
for i in epoch1:
    print("===========================Epoch: " +str(i) + "====================================")

    r_int_month = gk3[gk3['epoch'] == i]
    r_int_month=r_int_month.amount.tolist()[0]
    print(r_int_month)

    s_int_month = dk[dk['epoch'] == i]
    s_int_month=s_int_month.Actual_INDY_Staked.tolist()[0]
    print(s_int_month)
    
    
    
    inter = ((r_int_month/s_int_month)*73)
    print(inter)
    print("=======================================================================")
    APY3_arr.append(inter*100)
    print('APR: '+str(APY3_arr[0]))


===========================Epoch: 512====================================
6046.109993
8043703.5831120005
0.054870996292760134
APR: 5.487099629276013


**The mean APR for the month can also be calculated and is obtained as below:**

In [16]:
# monthly
epoch1=[502,503,504,505,506,507]#change this
APY3_arr=[]
running_sum1=0
gk3= gk3.reset_index()
dk= dk.reset_index()
for i in epoch1:
    print("===========================Epoch: " +str(i) + "============================================")

    r_int_month = gk3[gk3['epoch'] == i]
    r_int_month=r_int_month.amount.tolist()[0]
    print(r_int_month)

    
    s_int_month = dk[dk['epoch'] == i]
    s_int_month=s_int_month.Actual_INDY_Staked.tolist()[0]
    print(s_int_month)
    inter = ((r_int_month/s_int_month)*73)
    print(inter)
    APY3_arr.append(inter*100)
print("=======================================================================")
# print(APY3_arr)
print("Mean APR: "+str(sum(APY3_arr)/len(APY3_arr)))



===========================Epoch: 502============================================
6046.110016
8089002.636751001
0.0545637145873496
===========================Epoch: 503============================================
6046.110001
8100000.398709
0.054489630660184425
===========================Epoch: 504============================================
6046.110003
8082022.6888190005
0.054610837807917034
===========================Epoch: 505============================================
6046.110006999999
8137917.765994
0.05423574472026993
===========================Epoch: 506============================================
6046.109995
8033777.320347
0.05493879305282216
===========================Epoch: 507============================================
6046.110003
7938219.069132
0.055600132268365444
Mean APR: 5.473980884948477


**Now, in the below steps we'll calculate the ADA rewards earned by INDY stakers:**

In [17]:

# open a connection to a URL using urllib
webUrl  = urllib.request.urlopen('https://analytics.indigoprotocol.io/api/distributions')

# read the data from the URL and print it
data = webUrl.read()
# print (data)

# byteData 
b1 = str(data, 'UTF-8')  
# print(b1)
type(b1)

json_object = json.loads(b1)
# print(type(json_object))

df = pd.DataFrame(json_object)

REF_DATE: Final[datetime] = datetime(2022, 11, 27)
EPOCH: Final[int] = 379

def get_difference(date: datetime) -> int:
    difference = date - REF_DATE

    return difference.days

def func(value):
    full_date = time.strftime("%d %b %Y", time.gmtime(value))
    date = pd.to_datetime(full_date)
    diff = get_difference(date)
    return 379+(diff//5)
    
arr=[]
for i in range(len(df)):
    arr.append(func(df.loc[i, "timestamp"]))
#     df.loc[i, 2] = 181
df.insert(2, column="epoch" , value=arr)
df.insert(3, column="actual_ada_distributed", value=df['ada_distributed']/1000000)
df
columns_to_drop = ['indy_usd_price','ada_usd_price']

# Drop the specified columns
df = df.drop(columns=columns_to_drop)
df

df['indy_ada_price'] = df['indy_ada_price'].astype(float)
df.insert(6, column="actual_value_in_INDY", value=df['actual_ada_distributed']/df['indy_ada_price'])
dk2=df.groupby('epoch')
dk22=dk2.sum()
dk23=dk22.tail(10)
print(dk23.to_string())



             slot  ada_distributed  actual_ada_distributed  indy_ada_price    timestamp  actual_value_in_INDY
epoch                                                                                                        
504    1056706309      15877932111            15877.932111       11.556069  13789236637          11133.793116
505    1591806023        953874568              953.874568       18.259441  20690601515            629.702321
506    2659783124       1949910174             1949.910174       30.520843  34491108944           1274.561287
507    1467672849      15999384042            15999.384042       15.931183  18974902050          11050.742643
508    1472797055       1063230886             1063.230886       15.794985  18980026256            740.763991
509     940530964        463804227              463.804227        9.723886  12081495001            334.642973
510    1482018940      15769303161            15769.303161       14.500850  18989248141          11415.118762
511     54

**The epoch wise distrubution of ADA(in terms of INDY) is as follows:**

In [18]:
epochs=[508,509,510,511,512,507]

total=0
arr_profit_sharing=[]
# for i in arr_epochs:
print("Distribution of ADA in terms of INDY")
for i in epochs:
    epo=dk2.get_group(i).sum()
    value=epo.actual_value_in_INDY
    total=total+value
    print("Epoch " +str(i) +": "+str(value))
print("Total: " +str(total))

Distribution of ADA in terms of INDY
Epoch 508: 740.7639914306301
Epoch 509: 334.6429732818901
Epoch 510: 11415.118761725296
Epoch 511: 162.8661872955356
Epoch 512: 13591.660187680458
Epoch 507: 11050.742642933028
Total: 37295.79474434684


**The monthly mean amount Staked INDY and APR is obtained as follows:**

In [19]:
#monthly
import urllib.request
import pandas as pd
import json
import time
from datetime import datetime
from typing import Final
# open a connection to a URL using urllib
webUrl  = urllib.request.urlopen('https://analytics.indigoprotocol.io/api/staking-manager/history?page=1&perPage=1000')

# read the data from the URL and print it
data = webUrl.read()
# print (data)

# byteData = b'Lets eat a \xf0\x9f\x8d\x95!'  
b1 = str(data, 'UTF-8')  
# print(b1)
type(b1)
json_object = json.loads(b1)
data_staking=json_object['data']
df_staking = pd.DataFrame(data_staking)
df_staking["date"] = df_staking["timestamp"].str.split("T").str[0]
df_staking["date"] = pd.to_datetime(df_staking["date"])

df_filtered_staking = df_staking.loc[(df_staking['date'] >= '2024-08-28')
                     &(df_staking['date'] <= '2024-09-26')]

answer=(df_filtered_staking['totalStake'].sum()/(1000000*30))
print("Mean INDY staked: "+str(answer))
print("APR: "+str((total/answer)*12*100))

Mean INDY staked: 8757902.3174684
APR: 5.1102366834976625
